In [1]:
cd ..

C:\Users\User\Documents\SURF 2021\RL RNN SURF\src


We first define the input data. The input data is has the following structure: We have a group of subjects (say S1 and S2) and each subject has completed the decision-making task multiple times (multiple blocks), say each subject has completed the task two times in this example, i.e., we have two blocks of data for each subject. The data within each block is a ditonary containing three numpy arrays: 'action', 'state', 'reward'. 

'action' containig the actions taken by the subject on each trial and it should be a non-zero integer or -1. If the action is -1 it will coded by a zero vector and corresponds to no-action. Dimesionlity of 'action' is B x |T| in which |T| is the number of trials.

'state' contains the state of the environment each trial. Its dimesionlity is B x |T| x |S| in which |T| is the number of trials, and |S| is the lenght of state vector. 

'reward' contains the reward received after taking each actions. Its dimesionlity is B x |T| in which |T| is the number of trials.

For example, if subject S1 has completed 6 trials in the firt block and 4 trials in the second block and subject 2 has completed 5, 6 trials in the first and second blocks respectivly, then the data structure can look like this:

   

In [2]:
#  Data
import numpy as np
import scipy.io as spio
data = {}
test_data = {}
for i in range(1,101):
    testData = spio.loadmat('genData_smG_rlG\sub_'+str(i)+'.mat')
    struct = testData['subData']
    block = [
                {
                    'action': np.array([[resp-1 for ([id],[resp]) in zip(struct[0,0]['blockID'],struct[0,0]['selectStimIndex']) if id==blockID]]),
                    'state': np.array([[stimOffers for ([id],stimOffers) in zip(struct[0,0]['blockID'],struct[0,0]['stimOffers']) if id==blockID]]),
                    'reward': np.array([[outcomeRew for ([id],[outcomeRew]) in zip(struct[0,0]['blockID'],struct[0,0]['outcomeRew']) if id==blockID]]),
                    'id': 'S'+str(i),

                    'block': blockID
                } for blockID in np.unique([struct[0,0]['blockID']])

            ]
#     block = [
#                 {
#                     'action': np.array([struct[0,0]['selectStimIndex'].flatten()])-1,
#                     'state': np.array([struct[0,0]['stimOffers']]),
#                     'reward': np.array([struct[0,0]['outcomeRew'].flatten()]),
#                     'id': 'S'+str(i),

#                     'block': 0
#                 }

#             ]
    if i<91:
        data['S'+str(i)] = block
    else:
        test_data['S'+str(i)] = block

In [3]:
data

{'S1': [{'action': array([[2, 2, 0, 1, 2, 1, 0, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
           2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2,
           1, 1, 2, 2]]),
   'state': array([[[3, 1],
           [3, 2],
           [3, 1],
           [2, 1],
           [3, 1],
           [2, 1],
           [1, 3],
           [2, 1],
           [2, 3],
           [3, 1],
           [3, 1],
           [1, 2],
           [2, 3],
           [1, 2],
           [1, 2],
           [1, 2],
           [2, 1],
           [3, 2],
           [3, 2],
           [2, 3],
           [3, 1],
           [2, 3],
           [1, 3],
           [3, 2],
           [2, 1],
           [2, 1],
           [3, 2],
           [1, 3],
           [2, 1],
           [2, 1],
           [2, 3],
           [1, 2],
           [2, 3],
           [3, 2],
           [1, 3],
           [3, 1],
           [3, 2],
           [3, 1],
           [3, 2],
           [3, 1],
           [2, 1],
           [2, 3

In the above example, |A|=2 (there are two actions coded as 0 and 1), and |S|=2 (the state vector has two elements). For example, if there are three stimuli in the environment, they can be coded as [1, 0, 0], [0, 1, 0], [0, 0, 1] state vectors. In this case |S|=3.

In [4]:
from actionflow.rnn.lstm_beh import LSTMBeh
worker = LSTMBeh(a_size=3, s_size=2, n_cells=20)

c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\user\documents\surf

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


Finally, we train the model:

In [5]:
from actionflow.rnn.opt_beh import OptBEH
from actionflow.util.logger import LogFile

output_path = '../results/simpleModel_selectedStimulus/'
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, None,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None
                    )

2021-07-02 14:25:28,065 - DeepRL - DEBUG - version control: (None, None)
2021-07-02 14:25:28,067 - DeepRL - DEBUG - learning rate: 0.01
2021-07-02 14:25:28,067 - DeepRL - DEBUG - global iters: 50
2021-07-02 14:25:28,068 - DeepRL - DEBUG - training data-points: 90
2021-07-02 14:25:28,069 - DeepRL - DEBUG - test data-points: None


Instructions for updating:
Use tf.cast instead.


2021-07-02 14:25:29,010 - DeepRL - DEBUG - opt started...


INFO:tensorflow:../results/simpleModel_selectedStimulus/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 14:25:39,909 - DeepRL - DEBUG - global iter =    0 total obj: 16387.6026
2021-07-02 14:25:50,330 - DeepRL - DEBUG - global iter =    1 total obj: 9080.4123
2021-07-02 14:26:02,999 - DeepRL - DEBUG - global iter =    2 total obj: 6131.4128
2021-07-02 14:26:16,001 - DeepRL - DEBUG - global iter =    3 total obj: 5064.4287
2021-07-02 14:26:32,115 - DeepRL - DEBUG - global iter =    4 total obj: 4478.0368
2021-07-02 14:26:47,875 - DeepRL - DEBUG - global iter =    5 total obj: 4037.8645
2021-07-02 14:27:04,197 - DeepRL - DEBUG - global iter =    6 total obj: 3826.7766
2021-07-02 14:27:20,107 - DeepRL - DEBUG - global iter =    7 total obj: 3582.4998
2021-07-02 14:27:35,703 - DeepRL - DEBUG - global iter =    8 total obj: 3330.2640
2021-07-02 14:27:51,098 - DeepRL - DEBUG - global iter =    9 total obj: 3341.5163
2021-07-02 14:28:07,493 - DeepRL - DEBUG - global iter =   10 total obj: 3144.8155
2021-07-02 14:28:23,491 - DeepRL - DEBUG - global iter =   11 total obj: 2960.3599
202

INFO:tensorflow:../results/simpleModel_selectedStimulus/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [6]:
from actionflow.data.data_process import DataProcess
train_merged = DataProcess.merge_data(data)
train_merged

{'merged': [{'reward': array([[ 25,   0,   0, ...,  25,  25,   0],
          [  0, -25, -25, ...,   0,   0, -25],
          [ 25,   0,  25, ...,   0,  25,  25],
          ...,
          [  0, -25, -25, ...,   0,   0,   0],
          [ 25,   0,  25, ...,   0,  25,  25],
          [  0, -25,   0, ..., -25,   0,   0]], dtype=int16),
   'action': array([[2, 2, 0, ..., 0, 0, 0],
          [2, 2, 0, ..., 0, 0, 0],
          [0, 0, 2, ..., 1, 2, 2],
          ...,
          [2, 2, 0, ..., 1, 2, 2],
          [0, 0, 2, ..., 1, 2, 2],
          [0, 0, 2, ..., 1, 2, 2]]),
   'state': array([[[3, 1],
           [3, 2],
           [3, 1],
           ...,
           [2, 1],
           [1, 3],
           [3, 1]],
   
          [[3, 1],
           [3, 2],
           [3, 1],
           ...,
           [2, 1],
           [1, 3],
           [3, 1]],
   
          [[1, 2],
           [3, 1],
           [1, 3],
           ...,
           [2, 1],
           [1, 3],
           [1, 3]],
   
          ...,
  

And then the merged data can be used for training the model as before. The test data can also be passed to the training method, in order to test the mmodel on the training data in regular intervals. Say the test data is as follows:

and we want to test the model every 10 iterations:

In [7]:
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, test_data,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None,
                    test_period=10
                    )

2021-07-02 14:44:43,424 - DeepRL - DEBUG - version control: (None, None)
2021-07-02 14:44:43,426 - DeepRL - DEBUG - learning rate: 0.01
2021-07-02 14:44:43,427 - DeepRL - DEBUG - global iters: 50
2021-07-02 14:44:43,429 - DeepRL - DEBUG - training data-points: 90
2021-07-02 14:44:43,431 - DeepRL - DEBUG - test data-points: 10
2021-07-02 14:45:22,547 - DeepRL - DEBUG - opt started...
2021-07-02 14:45:22,550 - DeepRL - DEBUG - started testing...
2021-07-02 14:47:45,205 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/simpleModel_selectedStimulus/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 14:48:15,848 - DeepRL - DEBUG - global iter =    0 total obj: 16406.4032
2021-07-02 14:48:33,514 - DeepRL - DEBUG - global iter =    1 total obj: 11198.7369
2021-07-02 14:48:51,093 - DeepRL - DEBUG - global iter =    2 total obj: 8755.5937
2021-07-02 14:49:10,348 - DeepRL - DEBUG - global iter =    3 total obj: 7106.3498
2021-07-02 14:49:28,202 - DeepRL - DEBUG - global iter =    4 total obj: 5920.2033
2021-07-02 14:49:45,683 - DeepRL - DEBUG - global iter =    5 total obj: 5319.2498
2021-07-02 14:50:03,604 - DeepRL - DEBUG - global iter =    6 total obj: 4927.2318
2021-07-02 14:50:21,141 - DeepRL - DEBUG - global iter =    7 total obj: 4475.7947
2021-07-02 14:50:36,182 - DeepRL - DEBUG - global iter =    8 total obj: 4148.0225
2021-07-02 14:50:52,753 - DeepRL - DEBUG - global iter =    9 total obj: 3913.3078
2021-07-02 14:50:52,755 - DeepRL - DEBUG - started testing...
2021-07-02 14:52:24,750 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/simpleModel_selectedStimulus/model-10/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 14:52:55,943 - DeepRL - DEBUG - global iter =   10 total obj: 3602.3861
2021-07-02 14:53:12,967 - DeepRL - DEBUG - global iter =   11 total obj: 3486.4793
2021-07-02 14:53:30,145 - DeepRL - DEBUG - global iter =   12 total obj: 3281.8574
2021-07-02 14:53:46,210 - DeepRL - DEBUG - global iter =   13 total obj: 3205.0268
2021-07-02 14:54:05,117 - DeepRL - DEBUG - global iter =   14 total obj: 3202.1620
2021-07-02 14:54:23,005 - DeepRL - DEBUG - global iter =   15 total obj: 3017.1604
2021-07-02 14:54:40,187 - DeepRL - DEBUG - global iter =   16 total obj: 2977.0319
2021-07-02 14:54:57,389 - DeepRL - DEBUG - global iter =   17 total obj: 2910.1044
2021-07-02 14:55:14,768 - DeepRL - DEBUG - global iter =   18 total obj: 2961.8109
2021-07-02 14:55:29,481 - DeepRL - DEBUG - global iter =   19 total obj: 2832.2501
2021-07-02 14:55:29,483 - DeepRL - DEBUG - started testing...
2021-07-02 14:57:14,854 - DeepRL - DEBUG - finished testing.


...

INFO:tensorflow:../results/simpleModel_selectedStimulus/model-30/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 15:02:58,830 - DeepRL - DEBUG - global iter =   30 total obj: 2497.5799
2021-07-02 15:03:14,499 - DeepRL - DEBUG - global iter =   31 total obj: 2532.0459
2021-07-02 15:03:29,021 - DeepRL - DEBUG - global iter =   32 total obj: 2619.0989
2021-07-02 15:03:41,806 - DeepRL - DEBUG - global iter =   33 total obj: 2488.0473
2021-07-02 15:03:53,339 - DeepRL - DEBUG - global iter =   34 total obj: 2488.7886
2021-07-02 15:04:04,887 - DeepRL - DEBUG - global iter =   35 total obj: 2459.1931
2021-07-02 15:04:17,305 - DeepRL - DEBUG - global iter =   36 total obj: 2451.5305
2021-07-02 15:04:28,956 - DeepRL - DEBUG - global iter =   37 total obj: 2396.5491
2021-07-02 15:04:40,654 - DeepRL - DEBUG - global iter =   38 total obj: 2352.1131
2021-07-02 15:04:52,312 - DeepRL - DEBUG - global iter =   39 total obj: 2296.2958
2021-07-02 15:04:52,314 - DeepRL - DEBUG - started testing...
2021-07-02 15:07:23,388 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/simpleModel_selectedStimulus/model-40/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 15:07:42,784 - DeepRL - DEBUG - global iter =   40 total obj: 2273.2825
2021-07-02 15:07:52,289 - DeepRL - DEBUG - global iter =   41 total obj: 2281.9609
2021-07-02 15:08:03,562 - DeepRL - DEBUG - global iter =   42 total obj: 2289.0592
2021-07-02 15:08:14,934 - DeepRL - DEBUG - global iter =   43 total obj: 2332.7290
2021-07-02 15:08:25,548 - DeepRL - DEBUG - global iter =   44 total obj: 2331.8432
2021-07-02 15:08:35,418 - DeepRL - DEBUG - global iter =   45 total obj: 2190.1490
2021-07-02 15:08:46,533 - DeepRL - DEBUG - global iter =   46 total obj: 2252.4466
2021-07-02 15:08:57,710 - DeepRL - DEBUG - global iter =   47 total obj: 2227.4081
2021-07-02 15:09:08,843 - DeepRL - DEBUG - global iter =   48 total obj: 2322.4479
2021-07-02 15:09:20,171 - DeepRL - DEBUG - global iter =   49 total obj: 2069.5335
2021-07-02 15:09:20,173 - DeepRL - DEBUG - opt finished.
2021-07-02 15:09:20,174 - DeepRL - DEBUG - started testing...
2021-07-02 15:11:55,684 - DeepRL - DEBUG - finished

INFO:tensorflow:../results/simpleModel_selectedStimulus/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [8]:
test_data["S100"]

[{'action': array([[2, 2, 2, 0, 2, 2, 0, 2, 1, 0, 2, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 2,
          0, 2, 2, 1, 2, 0, 2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0,
          1, 1, 0, 0]]),
  'state': array([[[3, 2],
          [1, 3],
          [1, 3],
          [1, 2],
          [2, 3],
          [1, 3],
          [1, 2],
          [3, 2],
          [1, 2],
          [2, 1],
          [2, 3],
          [3, 2],
          [3, 1],
          [2, 3],
          [3, 2],
          [2, 1],
          [2, 1],
          [1, 3],
          [2, 1],
          [2, 1],
          [1, 3],
          [3, 2],
          [3, 1],
          [1, 3],
          [2, 3],
          [1, 2],
          [1, 3],
          [2, 1],
          [1, 3],
          [3, 1],
          [1, 2],
          [2, 3],
          [2, 1],
          [3, 1],
          [2, 1],
          [3, 2],
          [1, 2],
          [1, 2],
          [2, 3],
          [1, 3],
          [3, 2],
          [2, 3],
          [3, 1],
          [3, 1],
          [2